## Matching previously blocked dataset

Following the instructons you should have checked out the project and started jupyter notebook in the parent folder.
```
uni-sofia-entity-linking-magellan    <= "jupyter notebook" started here
 |- datasets
 |- notebooks
    |- entity_match_electronics.ipynb
```
 

In [200]:
# Import py_entitymatching package
import py_entitymatching as em
import os
import pandas as pd

In [201]:
# Read the CSV files and set 'ID' as the key attribute
A = em.read_csv_metadata("../dataset/amazon_new_features.csv", key='ID')
B = em.read_csv_metadata("../dataset/best_buy_new_features.csv", key='ID')
# Load the pre-labeled data
G = em.read_csv_metadata('../dataset/sample_blocked_500_labeled.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')

print (f"len(G)={len(G)}, labeled={G['label'].value_counts()[1]}")


Metadata file is not present in the given path; proceeding to read the csv file.


len(G)=500, labeled=45


In [202]:
G.head(3)

,Unnamed: 0,_id,ltable_ID,rtable_ID,ltable_Brand,ltable_Name,ltable_Amazon_Price,ltable_Original_Price,ltable_Features,ltable_Parameters,ltable_tfidf,rtable_Brand,rtable_Name,rtable_Price,rtable_Description,rtable_Features,rtable_Parameters,rtable_tfidf,label
0,4928,4928,4,124,HP,HP Chromebook 14 - New Version (Snow White),$339.99,NaN,"NVIDIA Tegra K1 Processor. 2 GB DDR3L SDRAM. 16 GB Solid-State Drive. 14-Inch Screen. Chrome, 8....",14 K1 2 DDR3L 16 14 8.0,new hp white chromebook 14,HP,HP Pavilion 15.6 Laptop AMD A10Series 8GB Memory 1TB Hard Drive Silver 15-ab161nr,$599.99,"Pavilion 15.6&#34; Laptop - AMD A10-Series - 8GB Memory - 1TB Hard Drive, Read customer reviews ...","Windows 10 operating system,15.6 display,AMD A10-8700P accelerated processor,8GB system memory f...",15.6 A10Series 8GB 1TB 15 ab161nr 15.6 A10 8GB 1TB 10 15.6 A10 8700P 8GB 1TB R6 4.89 0.99 2 3.0 ...,ab161nr a10series pavilion hp amd,0
1,6553,6553,7,570,Dell,"Dell Inspiron 11.6-Inch 2 in 1 Convertible Touchscreen Laptop, i3147-3750sLV",$371.14,$449.99,Intel Pentium Processor N3520 2.16GHz. 4 GB DDR3L SDRAM. 500 GB 7200 rpm Hard Drive. 11.6-Inch S...,11.6 2 1 i3147 3750sLV N3520 2.16GHz. 4 DDR3L 500 7200 11.6 8.1; 10,convertible inch dell inspiron 11,Dell,Dell Dell Latitude E5500 Laptop No Black E5500,$144.99,"Latitude E5500 Laptop Notebook-Core 2 Duo 2.0GHz-2GB DDR2-80GB-DVD+CDRW-Windows 7, Read customer...","Processor Manufacturer: Intel,Processor Type: Core 2 Duo,Processor Speed: 2 GHz,Processor Core: ...",E5500 E5500 E5500 2 2.0GHz 2GB DDR2 80GB 7 2 2 2 2 DDR2 80 15.4 16:10 4500MHD DDR2 802.11b 4 45 ...,e5500 dell latitude laptop black,0
2,7241,7241,8,408,Dell,"Super Power Supply? AC / DC Laptop Adapter Charger Cord for Dell XPS, Dell Inspiron P28F, P25F, ...",$18.89,NaN,Input: 110-220V Output: 19.5V 4.62A. Warranty: 1 Year. Super Power Supply? are manufactured and ...,P28F P25F P35G P17E P24G P30G P23F P04E M102z P19Ta P31G P37G P33G P37G P36F P26F P24E P15E 110 ...,plug cord dc netbook dell,Dell,Dell Inspiron 15.6 TouchScreen Laptop Intel Core i7 8GB Memory 1TB Hard Drive Silver I7558-4010BLK,$799.99,"Inspiron 15.6&#34; Touch-Screen Laptop - Intel Core i7 - 8GB Memory - 1TB Hard Drive, Read custo...","Microsoft Windows 8.1 operating system preinstalled,15.6 Full HD touch screen for hands-on contr...",15.6 i7 8GB 1TB I7558 4010BLK 15.6 i7 8GB 1TB 8.1 15.6 5th i7 5500U 8GB 1TB 4.8 0.78 4.0 2 3.0 1...,4010blk i7558 dell inspiron 1tb,0


## Splitting the labeled data into development and evaluation set


In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [203]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [204]:
print(f"I[label==1] = {len(I[I['label'] == 1])}")
print(f"J[label==1] = {len(J[J['label'] == 1])}")

J.head(3)

I[label==1] = 25
J[label==1] = 20


,Unnamed: 0,_id,ltable_ID,rtable_ID,ltable_Brand,ltable_Name,ltable_Amazon_Price,ltable_Original_Price,ltable_Features,ltable_Parameters,ltable_tfidf,rtable_Brand,rtable_Name,rtable_Price,rtable_Description,rtable_Features,rtable_Parameters,rtable_tfidf,label
90,720174,720174,550,423,Lenovo,Lenovo Flex 3 14 (80R3000UUS) 14.0\ Laptop,$831.33,$849.99,"Intel Core i7-6500U 2.5 GHz. 8 GB DDR3L SDRAM. 1000 GB 5400 rpm Hard Drive. 14.0-Inch Screen, In...",3 14 80R3000UUS 14.0 i7 6500U 2.5 8 DDR3L 1000 5400 14.0 10 5,flex lenovo 14 laptop zseries,Lenovo,Lenovo Flex 3 2in1 11.6 TouchScreen Laptop Intel Celeron 4GB Memory 500GB Hard Drive Black Flex ...,$349.99,"Flex 3 2-in-1 11.6&#34; Touch-Screen Laptop - Intel Celeron - 4GB Memory - 500GB Hard Drive, Rea...","Windows 10 operating system,11.6 LCD 10-point multitouch screen,Intel?? Celeron?? processor N284...",3 2in1 11.6 4GB 500GB 3 11 80LX0026US 3 2 1 11.6 4GB 500GB 10 11.6 10 N2840 4GB 500GB 360?? 3.05...,flex 80lx0026us 11 lenovo celeron,1
254,2804508,2804508,1821,1773,Other,BRINCH? [Red] Deluxe Universal Fabric Portable thin Light Durable Waterproof Anti-tear 15 - 15.6...,NaN,NaN,"Bag Interior Size:380*290*35mm ,Exterior Size:390*300*40mm,fits 15.6\ Notebook/Macbook,Accesorie...",15 15.6 15 15.6 Size:380 290 35mm Size:390 300 40mm 15.6 Size:135 165 30mm. 15.6 X551MA SX018H C...,inch bag soft pouch computers,Other,Platinum Laptop Tote Bag Black PT-TBC02,$79.99,"Laptop Tote Bag, Read customer reviews and buy online at Best Buy.","Fits most laptops with up to a 15.6 display,Pockets,Canvas material,Sleeve,For use with your exi...",TBC02 15.6,tbc02 pt platinum tote bag,0
283,3453837,3453837,2208,1997,Other,"13 Inch Laptop Sleeve, 13 Inch Laptop Shoulder Bag, KOZMICC 13 13.3 Inch Neoprene Messenger (Bla...",$16.99,$26.99,Fits most 13 - 13.3 inch laptops from Dell HP Lenovo Acer ASUS Chromebook Samsung Apple Macbook ...,13 13 13 13.3 13 13.3 12.75 9.75,inch 13 strap neoprene messenger,Dell,Dell 13.3 Latitude Notebook 4 GB Memory 160 GB Hard Drive Black E4300,$198.91,"13.3 Latitude Notebook - 4 GB Memory - 160 GB Hard Drive, Read customer reviews and buy online a...","Processor Manufacturer: Intel,Processor Type: Core 2 Duo,Processor Speed: 2.40 GHz,Processor Cor...",13.3 4 160 E4300 13.3 4 160 2 2.40 2 4 DDR3 160 13.3 16:9 1366 768 DDR3 2 45 7,e4300 160 dell gb 13,0


## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation


In [205]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

### Creating Features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [206]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [207]:
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,ID_ID_exm,ID,ID,None,None,exact_match,<function ID_ID_exm at 0x7fe963302dc0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,ID_ID_anm,ID,ID,None,None,abs_norm,<function ID_ID_anm at 0x7fe9639349d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,ID_ID_lev_dist,ID,ID,None,None,lev_dist,<function ID_ID_lev_dist at 0x7fe963934af0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,ID_ID_lev_sim,ID,ID,None,None,lev_sim,<function ID_ID_lev_sim at 0x7fe963934a60>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,Brand_Brand_jac_qgm_3_qgm_3,Brand,Brand,qgm_3,qgm_3,jaccard,<function Brand_Brand_jac_qgm_3_qgm_3 at 0x7fe963934b80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,Brand_Brand_cos_dlm_dc0_dlm_dc0,Brand,Brand,dlm_dc0,dlm_dc0,cosine,<function Brand_Brand_cos_dlm_dc0_dlm_dc0 at 0x7fe963f59e50>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,Brand_Brand_jac_dlm_dc0_dlm_dc0,Brand,Brand,dlm_dc0,dlm_dc0,jaccard,<function Brand_Brand_jac_dlm_dc0_dlm_dc0 at 0x7fe96392a040>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Brand_Brand_mel,Brand,Brand,None,None,monge_elkan,<function Brand_Brand_mel at 0x7fe96392a430>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,Brand_Brand_lev_dist,Brand,Brand,None,None,lev_dist,<function Brand_Brand_lev_dist at 0x7fe9633181f0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,Brand_Brand_lev_sim,Brand,Brand,None,None,lev_sim,<function Brand_Brand_lev_sim at 0x7fe9633185e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [208]:
# List the names of the features generated
feature_table['feature_name']

0                                     ID_ID_exm
1                                     ID_ID_anm
2                                ID_ID_lev_dist
3                                 ID_ID_lev_sim
4                   Brand_Brand_jac_qgm_3_qgm_3
5               Brand_Brand_cos_dlm_dc0_dlm_dc0
6               Brand_Brand_jac_dlm_dc0_dlm_dc0
7                               Brand_Brand_mel
8                          Brand_Brand_lev_dist
9                           Brand_Brand_lev_sim
10                              Brand_Brand_nmw
11                               Brand_Brand_sw
12            Features_Features_jac_qgm_3_qgm_3
13        Features_Features_cos_dlm_dc0_dlm_dc0
14        Parameters_Parameters_jac_qgm_3_qgm_3
15    Parameters_Parameters_cos_dlm_dc0_dlm_dc0
16                  tfidf_tfidf_jac_qgm_3_qgm_3
17              tfidf_tfidf_cos_dlm_dc0_dlm_dc0
18              tfidf_tfidf_jac_dlm_dc0_dlm_dc0
19                              tfidf_tfidf_mel
20                         tfidf_tfidf_l

### Converting the Development Set to  Feature Vectors

In [209]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='label',
                            show_progress=True) 

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [210]:
# Display first few rows
H.head(3)

,_id,ltable_ID,rtable_ID,ID_ID_exm,ID_ID_anm,ID_ID_lev_dist,ID_ID_lev_sim,Brand_Brand_jac_qgm_3_qgm_3,Brand_Brand_cos_dlm_dc0_dlm_dc0,Brand_Brand_jac_dlm_dc0_dlm_dc0,...,Parameters_Parameters_cos_dlm_dc0_dlm_dc0,tfidf_tfidf_jac_qgm_3_qgm_3,tfidf_tfidf_cos_dlm_dc0_dlm_dc0,tfidf_tfidf_jac_dlm_dc0_dlm_dc0,tfidf_tfidf_mel,tfidf_tfidf_lev_dist,tfidf_tfidf_lev_sim,tfidf_tfidf_nmw,tfidf_tfidf_sw,label
141,1209136,865,3259,0,0.265419,3,0.25,0.0,0.0,0.0,...,0.055641,0.136364,0.2,0.111111,0.551568,33,0.131579,0.0,12.0,0
383,4982595,3131,402,0,0.128393,4,0.00,1.0,1.0,1.0,...,0.295656,0.104167,0.4,0.250000,0.536951,16,0.407407,6.0,8.0,1
135,1147154,827,109,0,0.131802,3,0.00,1.0,1.0,1.0,...,0.129099,0.047619,0.2,0.111111,0.528042,27,0.228571,0.0,5.0,0


### Selecting the Best Matcher Using Cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [211]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='precision', random_state=0)
result['cv_stats']

/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedM

/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.261905,0.213333,0.228195
1,RF,0.200000,0.050000,0.080000
2,SVM,0.000000,0.000000,0.000000
3,LinReg,0.200000,0.050000,0.080000
4,LogReg,0.000000,0.000000,0.000000


In [212]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='recall', random_state=42)
result['cv_stats']

/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bobi/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedM

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.193333,0.27,0.21
1,RF,0.000000,0.00,0.00
2,SVM,0.000000,0.00,0.00
3,LinReg,0.000000,0.00,0.00
4,LogReg,0.000000,0.00,0.00


### Converting the Evaluation Set to  Feature Vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [213]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='label', show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


### Training the Selected Matcher

In [214]:
len(L)

150

In [215]:
# Train using feature vectors from I 
dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

### Predicting the Matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [216]:
# Predict on L 
predictions = dt.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

In [217]:
predictions[predictions['predicted']==1]

,_id,ltable_ID,rtable_ID,ID_ID_exm,ID_ID_anm,ID_ID_lev_dist,ID_ID_lev_sim,Brand_Brand_jac_qgm_3_qgm_3,Brand_Brand_cos_dlm_dc0_dlm_dc0,Brand_Brand_jac_dlm_dc0_dlm_dc0,...,tfidf_tfidf_jac_qgm_3_qgm_3,tfidf_tfidf_cos_dlm_dc0_dlm_dc0,tfidf_tfidf_jac_dlm_dc0_dlm_dc0,tfidf_tfidf_mel,tfidf_tfidf_lev_dist,tfidf_tfidf_lev_sim,tfidf_tfidf_nmw,tfidf_tfidf_sw,label,predicted
46,313856,239,714,0,0.334734,3,0.00,1.0,1.0,1.0,...,0.240741,0.2,0.111111,0.819797,15,0.531250,12.0,15.0,0,1
76,639367,484,526,0,0.920152,3,0.00,0.0,0.0,0.0,...,0.033898,0.2,0.111111,0.522976,28,0.125000,-3.0,5.0,0,1
327,4091604,2628,71,0,0.027017,4,0.00,1.0,1.0,1.0,...,0.096774,0.2,0.111111,0.589461,23,0.281250,5.0,14.0,0,1
142,1222684,872,319,0,0.365826,3,0.00,1.0,1.0,1.0,...,0.122807,0.2,0.111111,0.575543,24,0.250000,0.0,9.0,1,1
362,4669096,2948,2454,0,0.832429,3,0.25,1.0,1.0,1.0,...,0.204082,0.2,0.111111,0.684483,16,0.483871,10.0,11.0,0,1
219,2276742,1510,833,0,0.551656,4,0.00,1.0,1.0,1.0,...,0.117647,0.2,0.111111,0.520000,24,0.142857,1.0,9.0,1,1
6,27664,19,504,0,0.037698,3,0.00,1.0,1.0,1.0,...,0.145833,0.2,0.111111,0.658755,18,0.379310,4.0,7.0,1,1
113,935322,701,1374,0,0.510189,4,0.00,1.0,1.0,1.0,...,0.098361,0.2,0.111111,0.625964,29,0.147059,0.0,8.0,1,1
432,5744853,3569,2063,0,0.578033,3,0.25,1.0,1.0,1.0,...,0.100000,0.2,0.111111,0.500701,24,0.111111,0.0,7.0,1,1
430,5744739,3569,1693,0,0.474363,3,0.25,1.0,1.0,1.0,...,0.096154,0.2,0.111111,0.535109,25,0.137931,-1.0,7.0,0,1


### Evaluating the Matching Output

Finally, we evaluate the accuracy of predicted outputs

In [218]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 41.67% (5/12)
Recall : 25.0% (5/20)
F1 : 31.25%
False positives : 7 (out of 12 positive predictions)
False negatives : 15 (out of 138 negative predictions)
